# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

```pip install langchain-openai```

Accessing the API requires an API key, which you can get by creating an account and heading here. Once we have a key we'll want to set it as an environment variable by running:

```export OPENAI_API_KEY="..."```

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('paidDemo.csv')

In [ ]:
df.head()

# LLM Chain

#### Components

Import various components to be used

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

#### Create LLM reference and configuration

In [ ]:
llm_model = "gpt-3.5-turbo-0301"

llm = ChatOpenAI(temperature=0.9, model=llm_model)

# figure out how to get Gemini (or whatever) also referenced here, so I can switch them out easily

#### Chat Prompt Template

This uses template to create `prompt` with specific text, that will has a single parameter: **ValueProp**

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is a good name for company with the core value proposition: {ValueProp}?"
)
print(prompt)


#### Create a Chain with LLM and Prompt reference

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

#### Add input and run Chain

In [ ]:
valueProp = "revolutionary software library that unlocks the full potential of next-generation large language models like ChatGPT or Bard"
chain.run(valueProp)

In [ ]:
valueProp = "revolutionary software library that makes it super simple to analyze and translate test"
chain.run(valueProp)

In [ ]:
valueProp = "revolutionary software library that converts media between different formats"
chain.run(valueProp)

# SimpleSequentialChain

Simple chain where the outputs of one step feed directly into next.

#### Import Components

In [ ]:
from langchain.chains import SimpleSequentialChain

#### First Prompt - Create name of company

In [ ]:
first_prompt = ChatPromptTemplate.from_template(
    "What is a good name for company with the core value proposition: {ValueProp}?"
)

#Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

#### Second Chain - Use the generated name to write a description of the company

In [ ]:
#prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

#chain 2
chain_two = LLMChain(llm=llm, prompt= second_prompt)

##### Create a sequential chain to connect them

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
valueProp = "revolutionary software library that unlocks the full potential of next-generation large language models like ChatGPT or Bard"
overall_simple_chain.run(valueProp)

# SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
#prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English: {Review}"
)

#chain 1: input= Review, output= English_Review
chain_one = LLMChain(
    llm=llm, 
    prompt=first_prompt, 
    output_key="English_review"
)

In [ ]:
#prompt template 2: Summarize review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review into 1 sentence: {English_review}"
)

#chain 1: input= English_review, output= summary
chain_two = LLMChain(
    llm=llm, 
    prompt=second_prompt,
    output_key="summary"
)
    

In [ ]:
#prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review in: {Review}"
)

#chain 1: input= Review, output= language
chain_three = LLMChain(
    llm=llm, 
    prompt=third_prompt,
    output_key="language"
)

In [ ]:
#prompt template 4: follow up message
forth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language:\
    Summary: {summary}  Language: {language}"
)

#chain 4: input= suummary, language, output= followup_message
chain_four = LLMChain(
    llm=llm, 
    prompt=forth_prompt,
    output_key="followup_message"
)

In [ ]:
#overall_chain: input = Review output = English_review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_review", "summary", "followup_message"],
    verbose=True
)

In [ ]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
overall_chain(review)

# Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template= p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [ ]:
# Change this prompt, and notice that the route is different based on the type of prompt
chain.run("what is the square root of 48")